<a href="https://colab.research.google.com/github/akansh12/-Deep-Learning-for-Computer-Vision/blob/main/Part_A_Akansh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
import numpy as np
import torchvision
import os
import torch.nn as nn
from torch.nn import functional as F

In [2]:
import matplotlib.pyplot as plt

### Qurstion-1

In [3]:
### HELPER FUNCTIONS
def findConv2dOutShape(H_in,W_in,conv,pool=2):
  kernel_size=conv.kernel_size
  stride=conv.stride
  padding=conv.padding
  dilation=conv.dilation

  H_out=np.floor((H_in+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
  W_out=np.floor((W_in+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)
  if pool:
    H_out/=pool
    W_out/=pool
  return int(H_out),int(W_out)

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = "cpu"

In [34]:
class conv_net(nn.Module):
  def __init__(self, params): #num_filters, filter_size, activation_functions,dense_neurons,num_outputs
    super(conv_net,self).__init__()
    c_in, h_in, w_in = params["input_shape"]
    num_filters = params["num_filters"]
    filter_size = params["filter_size"]
    self.act_function = params["activation_functions"]
    dense_neurons = params["dense_neurons"]
    num_output = params["num_output"]
    self.dropout = params["dropout"]


    ####Conv layers
    self.conv1 = nn.Conv2d(c_in, num_filters[0], kernel_size=filter_size[0])
    h,w = findConv2dOutShape(h_in, w_in, self.conv1)

    self.conv2 = nn.Conv2d(num_filters[0], num_filters[1], kernel_size=filter_size[1])
    h,w = findConv2dOutShape(h, w, self.conv2)

    self.conv3 = nn.Conv2d(num_filters[1], num_filters[2], kernel_size=filter_size[2])
    h,w = findConv2dOutShape(h, w, self.conv3)


    self.conv4 = nn.Conv2d(num_filters[2], num_filters[3], kernel_size=filter_size[3])
    h,w = findConv2dOutShape(h, w, self.conv4)


    self.conv5 = nn.Conv2d(num_filters[3], num_filters[4], kernel_size=filter_size[4])
    h,w = findConv2dOutShape(h, w, self.conv5)
    print(h)
    print(w)
    #FC layers
    self.num_flatten = h*w*num_filters[4]
    print(self.num_flatten)
    self.fc1 = nn.Linear(self.num_flatten, dense_neurons)
    self.fc2 = nn.Linear(dense_neurons, num_output)

  def forward(self,x):
    x = self.act_function[0](self.conv1(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[1](self.conv2(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[2](self.conv3(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[3](self.conv4(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[4](self.conv5(x))
    x = F.max_pool2d(x, 2, 2)
    #Flatening the layers
    x = x.view(-1, self.num_flatten)
    
    x = self.act_function[5](self.fc1(x))
    x = F.dropout(x, self.dropout, training= self.training)
    x = self.fc2(x)
    x = F.log_softmax(x, dim=1)
    return x   


In [38]:
params = {"input_shape": (3,224,224),
          "num_filters": [8,16,32,64,128],
          "filter_size":[3,3,3,3,3],
          "activation_functions": [nn.functional.relu]*6,
          "dense_neurons": 128,
          "num_output":10,
          "dropout": 0.2          
          }

In [39]:
model = conv_net(params)
model=model.to(device)

5
5
3200


In [41]:
from torchsummary import summary
summary(model, input_size=(3, 224, 224),device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 222, 222]             224
            Conv2d-2         [-1, 16, 109, 109]           1,168
            Conv2d-3           [-1, 32, 52, 52]           4,640
            Conv2d-4           [-1, 64, 24, 24]          18,496
            Conv2d-5          [-1, 128, 10, 10]          73,856
            Linear-6                  [-1, 128]         409,728
            Linear-7                   [-1, 10]           1,290
Total params: 509,402
Trainable params: 509,402
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 5.50
Params size (MB): 1.94
Estimated Total Size (MB): 8.02
----------------------------------------------------------------


In [39]:
a = plt.imread("/content/drive/MyDrive/Nature_data/inaturalist_12K/train/Amphibia/0012ec13b97dfbfb3dd5de8c3da95555.jpg")

In [14]:
a = np.random.randn(96,96,3)

In [16]:
a_t = torch.from_numpy(a)

In [19]:
model(a_t.to(device))

NotImplementedError: ignored